# Introduction to SQLAlchemy
***

The purpose of these laboratory classes is to familiarize with the basic techniques of working with [SQLAlchemy](https://www.sqlalchemy.org/).

The scope of this classes:
- connection to a database (especially remote database),
- explore of database structure using ORM,
- creating simple select in ORM,
- using query results in a program,
- adding where clause to query.

SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language.

Object Relational Mapper (ORM) is a programming technique for converting data between incompatible type systems using object-oriented programming languages. This creates, in effect, a "virtual object database" that can be used from within the programming language.

The database structure is not always known to the programmer for various reasons, most often due to lack of documentation. In this case, it's worth knowing that we can use SQL query to explore database structure.

## Exercise 1 - PostgreSQL
***

### 1. Connection SQLAlchemy with database

To connection a database with a program with the basic version we need to use the following script:

In [1]:
from sqlalchemy import create_engine

config_PostgreSQL = {
    "database_type": "",
    "user": "",
    "password": "",
    "database_url": "",
    "port": ,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_PostgreSQL)

db = create_engine(db_string)

# test the connection
try:
    conn = db.connect()
    print("Connected successfully!")
except:
    print("Failed to connect")

Connected successfully!


where:
- *databae_type* is the name of database driver, more details you can read [here](https://docs.sqlalchemy.org/en/13/core/engines.html#database-urls),
- *user* - name of user in database,
- *password* - password to database,
- *database_url* - url address to database,
- *port* - port to database connection,
- *database_name* - name of database.  

*[create_engine](https://docs.sqlalchemy.org/en/13/core/connections.html)* is just interface to connection database with program.

For PostgreSQL database structure are describe [here](https://www.postgresqltutorial.com/postgresql-sample-database/):

![schema dvd rental](images/dvd-rental-sample-database-diagram.png)

### 2. Based on *information_schema*, present how to explore relationships between the tables

In all database exist metadata to describe structure of data. This data are write in *information_schema*, for PostgreSql [see](https://www.postgresql.org/docs/12/information-schema.html):

```sql
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'name' AND table_schema = 'public';
```

Present how to explore database structure between the tables:
1. staff and country
2. actor, language and film

Of course, SQLAlchemy has the function to read this data. In this sript *table_name* is the string with table name in database:

In [2]:
from sqlalchemy import MetaData, Table, inspect

inspection = inspect(db)
print("Tables in database:")
print(inspection.get_table_names())

metadata = MetaData()

# Define tables name
category = Table('category', metadata, autoload_with=db)
staff = Table('staff', metadata, autoload_with=db)
country = Table('country', metadata, autoload_with=db)

# Function to display column details
def display_column_details(table):
    print("="*50)
    print(f"Table: {table.name}")
    print("="*50)
    for column in table.columns:
        print("Column: {}, Type: {}, Nullable: {}, Primary Key: {}, Unique: {}".format(
            column.name, column.type, column.nullable, column.primary_key, column.unique))
    print(f"Columns keys: {table.columns.keys()}")
    print("="*50 + "\n")

# Display details of each column 'staff'
display_column_details(staff)

# Display details of each column 'country'
display_column_details(country)

Tables in database:
['country', 'film', 'address', 'authors', 'film_category', 'alfabet', 'countryies', 'cities', 'city_data', 'customer', 'countries', 'cityies', 'actor', 'staff', 'inventory', 'res', 'payment', 'monuments', 'language', 'store', 'shape_a', 'shape_b', 'users', 'books', 'category', 'film_actor', 'city', 'rental']
Table: staff
Column: staff_id, Type: INTEGER, Nullable: False, Primary Key: True, Unique: None
Column: first_name, Type: VARCHAR(45), Nullable: False, Primary Key: False, Unique: None
Column: last_name, Type: VARCHAR(45), Nullable: False, Primary Key: False, Unique: None
Column: address_id, Type: SMALLINT, Nullable: False, Primary Key: False, Unique: None
Column: email, Type: VARCHAR(50), Nullable: True, Primary Key: False, Unique: None
Column: store_id, Type: SMALLINT, Nullable: False, Primary Key: False, Unique: None
Column: active, Type: BOOLEAN, Nullable: False, Primary Key: False, Unique: None
Column: username, Type: VARCHAR(16), Nullable: False, Primary Ke

Basic query *select* in SQLAlchemy > 2.0 has form:

In [3]:
from sqlalchemy import text

# Define the statement
stmt = 'SELECT * FROM category'
con = db.connect()

# Execute the statement and fetch the results
results = con.execute(text(stmt)).fetchall()
# Print results from query
for row in results:
    print(row)

(1, 'Action', datetime.datetime(2006, 2, 15, 9, 46, 27))
(2, 'Animation', datetime.datetime(2006, 2, 15, 9, 46, 27))
(3, 'Children', datetime.datetime(2006, 2, 15, 9, 46, 27))
(4, 'Classics', datetime.datetime(2006, 2, 15, 9, 46, 27))
(5, 'Comedy', datetime.datetime(2006, 2, 15, 9, 46, 27))
(6, 'Documentary', datetime.datetime(2006, 2, 15, 9, 46, 27))
(7, 'Drama', datetime.datetime(2006, 2, 15, 9, 46, 27))
(8, 'Family', datetime.datetime(2006, 2, 15, 9, 46, 27))
(9, 'Foreign', datetime.datetime(2006, 2, 15, 9, 46, 27))
(10, 'Games', datetime.datetime(2006, 2, 15, 9, 46, 27))
(11, 'Horror', datetime.datetime(2006, 2, 15, 9, 46, 27))
(12, 'Music', datetime.datetime(2006, 2, 15, 9, 46, 27))
(13, 'New', datetime.datetime(2006, 2, 15, 9, 46, 27))
(14, 'Sci-Fi', datetime.datetime(2006, 2, 15, 9, 46, 27))
(15, 'Sports', datetime.datetime(2006, 2, 15, 9, 46, 27))
(16, 'Travel', datetime.datetime(2006, 2, 15, 9, 46, 27))


Function *execute* make a request to a database and *fetchall* method get our results from an executed query. But in this case we don't use ORM propertis. More correctly is use structur:

In [4]:
from sqlalchemy import select

staff = Table('staff', metadata, autoload_with=db)
address = Table('address', metadata, autoload_with=db)
city = Table('city', metadata, autoload_with=db)
country = Table('country', metadata, autoload_with=db)

# Create a select query with join
stmt2 = select(
    *[
        staff.c.staff_id, staff.c.first_name, staff.c.last_name,
        country.c.country_id, country.c.country
    ]
).select_from(
    staff.join(address, staff.c.address_id == address.c.address_id)
         .join(city, address.c.city_id == city.c.city_id)
         .join(country, city.c.country_id == country.c.country_id)
    )
# Explore relationships between tables (Staff and Country)
print(stmt2)

# Execute the statement and fetch the results
results = con.execute(stmt2).fetchmany(size=10)
# Print the SQL query string
print("Staff and Country:")
for row in results:
    print(row)

SELECT staff.staff_id, staff.first_name, staff.last_name, country.country_id, country.country 
FROM staff JOIN address ON staff.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id
Staff and Country:
(1, 'Mike', 'Hillyer', 20, 'Canada')
(2, 'Jon', 'Stephens', 8, 'Australia')


In [5]:
from sqlalchemy import select

film = Table('film', metadata, autoload_with=db)
language = Table('language', metadata, autoload_with=db)
actor = Table('actor', metadata, autoload_with=db)
film_actor = Table('film_actor', metadata, autoload_with=db)

stmt3 = select(
    *[
        film.c.film_id, film.c.title, film.c.description, film.c.release_year,
        language.c.language_id, language.c.name,
        actor.c.actor_id, actor.c.first_name, actor.c.last_name
    ]
).select_from(
    film.join(language, film.c.language_id == language.c.language_id)
        .join(film_actor, film.c.film_id == film_actor.c.film_id)
        .join(actor, film_actor.c.actor_id == actor.c.actor_id)
)
print(stmt3)

# Explore relationships between tables (Staff and Country)
con = db.connect()
# Execute the statement and fetch the results
results = con.execute(stmt3).fetchmany(size=10)
# Print the SQL query string
print("Actor, language and film:")
for row in results:
    print(row)

SELECT film.film_id, film.title, film.description, film.release_year, language.language_id, language.name, actor.actor_id, actor.first_name, actor.last_name 
FROM film JOIN language ON film.language_id = language.language_id JOIN film_actor ON film.film_id = film_actor.film_id JOIN actor ON film_actor.actor_id = actor.actor_id
Actor, language and film:
(1, 'Academy Dinosaur', 'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies', 2006, 1, 'English             ', 1, 'Penelope', 'Guiness')
(23, 'Anaconda Confessions', 'A Lacklusture Display of a Dentist And a Dentist who must Fight a Girl in Australia', 2006, 1, 'English             ', 1, 'Penelope', 'Guiness')
(25, 'Angels Life', 'A Thoughtful Display of a Woman And a Astronaut who must Battle a Robot in Berlin', 2006, 1, 'English             ', 1, 'Penelope', 'Guiness')
(106, 'Bulworth Commandments', 'A Amazing Display of a Mad Cow And a Pioneer who must Redeem a Sumo Wrestler in The Outback

### 3. Quantity categories of films are in the rental

In [6]:
# Define other columns for the category table if necessary
stmt4 = select(category)
print(stmt4)

# Execute the statement and fetch the results
results = con.execute(stmt4).fetchall()
# Print the SQL query string
print("\nNumber of categories in the rental:", len(results))

SELECT category.category_id, category.name, category.last_update 
FROM category

Number of categories in the rental: 16


### 4. Display list of categories with limit 2

In [7]:
stmt5 = select(category.columns.name).limit(2)
print(stmt5)

results = con.execute(stmt5).fetchall()
print("\nCategories with limit 2:")
for row in results:
    print(row)

SELECT category.name 
FROM category
 LIMIT :param_1

Categories with limit 2:
('Action',)
('Animation',)


### 5. Find the oldest and youngest film in rental

In [8]:
film = Table('film', metadata, autoload_with=db)

stmt6 = select(
    *[
        film.columns.title, film.columns.release_year
    ]
).order_by(film.columns.release_year.asc())
print(stmt6)

oldest_film = con.execute(stmt6).fetchone()


stmt7 = select(
    *[
        film.columns.title, film.columns.release_year
    ]
).order_by(film.columns.release_year.desc())
print(stmt7)

youngest_film = con.execute(stmt7).fetchone()

print("\nOldest film in rental:", oldest_film)
print("Youngest film in rental:", youngest_film)

SELECT film.title, film.release_year 
FROM film ORDER BY film.release_year ASC
SELECT film.title, film.release_year 
FROM film ORDER BY film.release_year DESC

Oldest film in rental: ('Chamber Italian', 2006)
Youngest film in rental: ('Chamber Italian', 2006)


### 6. Find all actor with a name: Olympia, Julia, Ellen. How can you check correction of your query

In [9]:
actor = Table('actor', metadata, autoload_with=db)

# define actors we want to find
actors_to_find = ['Olympia', 'Julia', 'Ellen']

# create select statement to find actors
stmt8 = select(
    *[
        actor
    ]
).where(actor.columns.first_name.in_(actors_to_find))
print(stmt8)

# execute statement and fetch results
results = con.execute(stmt8).fetchall()
# print results
print("\nActors with a name:")
for result in results:
    print(result.first_name, result.last_name)

SELECT actor.actor_id, actor.first_name, actor.last_name, actor.last_update 
FROM actor 
WHERE actor.first_name IN (__[POSTCOMPILE_first_name_1])

Actors with a name:
Julia Mcqueen
Julia Barrymore
Ellen Presley
Olympia Pfeiffer
Julia Zellweger
Julia Fawcett


## Exercise 2 - MySQL
***

### 1. Connection SQLAlchemy with database (MySQL)

To connection a database with a program with the basic version we need to use the following script:

In [10]:
from sqlalchemy import create_engine

config_MySQL = {
    "database_type": "",
    "user": "",
    "password": "",
    "database_url": "",
    "port": ,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_MySQL)

db = create_engine(db_string)

# test the connection
try:
    conn = db.connect()
    print("Connected successfully!")
except:
    print("Failed to connect")

Connected successfully!


where:
- *databae_type* is the name of database driver, more details you can read [here](https://docs.sqlalchemy.org/en/13/core/engines.html#database-urls),
- *user* - name of user in database,
- *password* - password to database,
- *database_url* - url address to database,
- *port* - port to database connection,
- *database_name* - name of database.  

*[create_engine](https://docs.sqlalchemy.org/en/13/core/connections.html)* is just interface to connection database with program.

For MySQL database structure are describe [here](https://www.mysqltutorial.org/mysql-sample-database.aspx):

![mysql schema database](images/mysql-sample-database.png)

### 2. Based on *information_schema*, present how to explore relationships between the tables

In all database exist metadata to describe structure of data. This data are write in *information_schema*, for MySQL [see](https://dev.mysql.com/doc/refman/8.0/en/getting-information.html):

```sql
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'bauer1'
```

Present how to explore database structure between the tables:
1. customers and employees
2. customers, payments and orders

Of course, SQLAlchemy has the function to read this data. In this sript *table_name* is the string with table name in database:

In [11]:
from sqlalchemy import MetaData, Table, inspect

inspection = inspect(db)
print("Tables in database:")
print(inspection.get_table_names())

metadata = MetaData()

# Define tables name
customers = Table('customers', metadata, autoload_with=db)
employees = Table('employees', metadata, autoload_with=db)
payments = Table('payments', metadata, autoload_with=db)
orders = Table('orders', metadata, autoload_with=db)

# Function to display column details
def display_column_details(table):
    print("="*50)
    print(f"Table: {table.name}")
    print("="*50)
    for column in table.columns:
        print("Column: {}, Type: {}, Nullable: {}, Primary Key: {}, Unique: {}".format(
            column.name, column.type, column.nullable, column.primary_key, column.unique))
    print(f"Columns keys: {table.columns.keys()}")
    print("="*50 + "\n")

display_column_details(customers)
display_column_details(employees)
display_column_details(payments)
display_column_details(orders)

Tables in database:
['bookings', 'cities', 'countries', 'customers', 'employees', 'hosts', 'offices', 'orderdetails', 'orders', 'payments', 'places', 'posts', 'productlines', 'products', 'reviews', 'users']
Table: customers
Column: customerNumber, Type: INTEGER, Nullable: False, Primary Key: True, Unique: None
Column: customerName, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: contactLastName, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: contactFirstName, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: phone, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: addressLine1, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: addressLine2, Type: VARCHAR(50), Nullable: True, Primary Key: False, Unique: None
Column: city, Type: VARCHAR(50), Nullable: False, Primary Key: False, Unique: None
Column: state, Type: VARCHAR(50), Nullable: True, Primary Ke

In [12]:
from sqlalchemy import Table, select, func

# Define relationships
stmt = select(
    *[
        customers, employees
    ]
).where(customers.columns.salesRepEmployeeNumber == employees.columns.employeeNumber)
print(stmt)

con = db.connect()
results = con.execute(stmt).fetchall()
# Print the SQL query string
print("Customers and employees:")
for result in results:
    print(result)

SELECT customers."customerNumber", customers."customerName", customers."contactLastName", customers."contactFirstName", customers.phone, customers."addressLine1", customers."addressLine2", customers.city, customers.state, customers."postalCode", customers.country, customers."salesRepEmployeeNumber", customers."creditLimit", employees."employeeNumber", employees."lastName", employees."firstName", employees.extension, employees.email, employees."officeCode", employees."reportsTo", employees."jobTitle" 
FROM customers, employees 
WHERE customers."salesRepEmployeeNumber" = employees."employeeNumber"
Customers and employees:
(124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '4155551450', '5677 Strong St.', None, 'San Rafael', 'CA', '97562', 'USA', 1165, Decimal('210500.00'), 1165, 'Jennings', 'Leslie', 'x3291', 'ljennings@classicmodelcars.com', '1', 1143, 'Sales Rep')
(129, 'Mini Wheels Co.', 'Murphy', 'Julie', '6505555787', '5557 North Pendale Street', None, 'San Francisco', 'CA', '

In [13]:
stmt = select(
    *[
        customers, payments, orders
    ]
).where(customers.columns.customerNumber == orders.columns.customerNumber
).where(orders.columns.orderNumber == payments.columns.checkNumber)
print(stmt)

results = con.execute(stmt).fetchall()
print("Customers, payments and orders:")
display(results)

SELECT customers."customerNumber", customers."customerName", customers."contactLastName", customers."contactFirstName", customers.phone, customers."addressLine1", customers."addressLine2", customers.city, customers.state, customers."postalCode", customers.country, customers."salesRepEmployeeNumber", customers."creditLimit", payments."customerNumber" AS "customerNumber_1", payments."checkNumber", payments."paymentDate", payments.amount, orders."orderNumber", orders."orderDate", orders."requiredDate", orders."shippedDate", orders.status, orders.comments, orders."customerNumber" AS "customerNumber_2" 
FROM customers, payments, orders 
WHERE customers."customerNumber" = orders."customerNumber" AND orders."orderNumber" = payments."checkNumber"
Customers, payments and orders:


[]

### 3. Number of products in the store

In [14]:
products = Table('products', metadata, autoload_with=db)

stmt = select(
    *[
        func.count(products.columns.productCode)
    ]
)
print(stmt)

result = con.execute(stmt).fetchone()
print(f"Number of products in the store: {result[0]}")

SELECT count(products."productCode") AS count_1 
FROM products
Number of products in the store: 110


### 4. List of offices with limit 5

In [15]:
offices = Table('offices', metadata, autoload_with=db)

stmt = select(*[offices])
stmt = stmt.limit(5)
print(stmt)

results = con.execute(stmt).fetchall()
for result in results:
    print(result)

SELECT offices."officeCode", offices.city, offices.phone, offices."addressLine1", offices."addressLine2", offices.state, offices.country, offices."postalCode", offices.territory 
FROM offices
 LIMIT :param_1
('1', 'San Francisco', '+1 650 219 4782', '100 Market Street', 'Suite 300', 'CA', 'USA', '94080', 'NA')
('2', 'Boston', '+1 215 837 0825', '1550 Court Place', 'Suite 102', 'MA', 'USA', '02107', 'NA')
('3', 'NYC', '+1 212 555 3000', '523 East 53rd Street', 'apt. 5A', 'NY', 'USA', '10022', 'NA')
('4', 'Paris', '+33 14 723 4404', "43 Rue Jouffroy D'abbans", None, None, 'France', '75017', 'EMEA')
('5', 'Tokyo', '+81 33 224 5000', '4-1 Kioicho', None, 'Chiyoda-Ku', 'Japan', '102-8578', 'Japan')


### 5. Find the oldest and youngest payments in rental

In [16]:
from sqlalchemy import Table, select, func

payments = Table('payments', metadata, autoload_with=db)

stmt = select(*[func.min(payments.columns.paymentDate),
                func.max(payments.columns.paymentDate)])
# stmt = stmt.where(payments.columns.rental_id.isnot(None))
print(stmt)

result = con.execute(stmt).fetchone()
print('Oldest payment date:', result[0])
print('Youngest payment date:', result[1])

SELECT min(payments."paymentDate") AS min_1, max(payments."paymentDate") AS max_1 
FROM payments
Oldest payment date: 2003-01-16
Youngest payment date: 2005-06-09
